In [115]:
import pandas as pd
import os

import matplotlib.pyplot as plt

main_path = "/Users/saurabhnagarkar/BLAH9/MultiSynDS/data/1_original/txt"

Loading the clinical cases

In [117]:
filename = os.path.join(main_path, "25350173.txt")
content = open(filename, "r").read()

content


'A 64-year-old female lifelong smoker with no previous cardiac history presented with a 4-day history of dyspnoea on minimal exertion (New York Heart Association class III). Electrocardiography showed broad complex monophasic ventricular tachycardia (VT) and troponin T was elevated. She was treated initially with intravenous amiodarone and adenosine with no sustained effect.\n\nTransthoracic echocardiography showed a severely dilated left ventricle (LV) with a large aneurysm and severely impaired ventricular function. The ejection fraction was less than 10%. Coronary angiography showed a long segment of moderate to severe disease in the left coronary system and a small right coronary artery with no flow limiting disease. Recurrent intermittent episodes of VT became persistent and intractable. Direct current cardioversion was attempted on at least 20 separate occasions, resulting in only transient amelioration of arrhythmia. Percutaneous coronary intervention was undertaken successfully

In [118]:
n_cc = len(os.listdir(main_path))
print("There are {} clinical cases".format(n_cc))

There are 1000 clinical cases


In [119]:
def extract_txt(path, filename):
    total_path = os.path.join(path, filename)
    return filename, open(total_path, "r").read()

filename = "25350173.txt"

def files_to_df(path, extensions=["txt"]):
    files = [x for x in os.listdir(path) if x.split(".")[-1] in extensions]
    data = [extract_txt(path, f) for f in files]
    return pd.DataFrame(data, columns=["filename", "text"])

df = files_to_df(main_path)

In [120]:
df.head()

,filename,text
0,32687096.txt,"A 13-year-old female patient (height 153 cm, w..."
1,37828735_5.txt,"Patient 5 was a 62-year-old man with ASD, mode..."
2,37272152.txt,"A 45-year-old male, without any previous cardi..."
3,33832121.txt,A 73-year-old man with ischemic heart disease ...
4,36055898.txt,We present the case of a 22-year-old female pa...


In [121]:
clinical_cases = df.apply(lambda row: row['text'], axis=1)
print(clinical_cases)

0      A 13-year-old female patient (height 153 cm, w...
1      Patient 5 was a 62-year-old man with ASD, mode...
2      A 45-year-old male, without any previous cardi...
3      A 73-year-old man with ischemic heart disease ...
4      We present the case of a 22-year-old female pa...
                             ...                        
995    A 53-year-old male presented to our outpatient...
996    A 49-year-old woman presented to the emergency...
997    History of presentation\n\nA 48-year-old man p...
998    Herein, we present the case of a 67‐year‐old f...
999    A 72 years old man, affected by post‐ischemic ...
Length: 1000, dtype: object


Loading the discharge summaries

In [122]:
main_path_discharge = "/Users/saurabhnagarkar/BLAH9/MultiSynDS/data/2_generated/2step_transformation_dt4h_GPT4omini"
lang = "en"
main_path_discharge = os.path.join(main_path_discharge, lang)


In [123]:
n_cc_discharge = len(os.listdir(main_path_discharge))
print("There are {} clinical cases".format(n_cc))

There are 1000 clinical cases


In [124]:
def extract_txt(path, filename):
    total_path = os.path.join(path, filename)
    return filename, open(total_path, "r").read()

def files_to_df(path, extensions=["txt"]):
    files = [x for x in os.listdir(path) if x.split(".")[-1] in extensions]
    data = [extract_txt(path, f) for f in files]
    return pd.DataFrame(data, columns=["filename", "text"])

df_discharge = files_to_df(main_path_discharge)

In [125]:
df_discharge.head()

,filename,text
0,31488018_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
1,31674934_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
2,31025656_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
3,29554788_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...
4,35502126_transformed_step1.txt,**Discharge Summary**\n\n**Patient Information...


In [141]:
discharge_summaries = df_discharge.apply(lambda row: row['text'], axis=1)
print(discharge_summaries)

0      **Discharge Summary**\n\n**Patient Information...
1      **Discharge Summary**\n\n**Patient Information...
2      **Discharge Summary**\n\n**Patient Information...
3      **Discharge Summary**\n\n**Patient Information...
4      **Discharge Summary**\n\n**Patient Information...
                             ...                        
995    **Discharge Summary**\n\n**Patient Information...
996    **Discharge Summary**\n\n**Patient Information...
997    **Discharge Summary**\n\n**Patient Information...
998    **Discharge Summary**\n\n**Patient Information...
999    **Discharge Summary**\n\n**Patient Information...
Length: 1000, dtype: object


In [127]:
import pandas as pd


In [128]:
with open("/Users/saurabhnagarkar/BLAH9/MultiSynDS/utils/prompts/guidelines.txt", "r") as file:
    guidelines = file.read()
#data is used to store the guidelines for the clinical cases

In [133]:
with open("/Users/saurabhnagarkar/BLAH9/MultiSynDS/utils/prompts/zero_shot.txt", "r") as file:
    zero_shot = file.read()

In [150]:
print(clinical_cases[0])

A 13-year-old female patient (height 153 cm, weight 37 kg, body surface area 1.21 m2) with a history of dilated cardiomyopathy who was on home Milrinone presented with increasing shortness of breath. Her worsening heart failure led to a planned LVAD (Jarvik 2000) implantation as a bridge to transplantation. Preoperative evaluation showed her hematological and biochemical values were within normal limits. Cardiac magnetic resonance imaging confirmed dilation of all cardiac chambers with left ventricle end diastolic volume 181 ml, end systolic volume 136 ml, stroke volume 45.2 ml, and ejection fraction 25%. Cardiac catheterization reports revealed: Cardiac output 2.5 l/min, systemic vascular resistance 2045 dynes s/cm5, pulmonary vascular resistance 384 dynes s/cm5, transpulmonary gradient 12 mmHg, and pulmonary artery pulsatility index 2.63.

Conventional monitors and a radial arterial catheter for blood pressure monitoring were applied while the patient was awake. Anesthetic induction 

In [156]:
import json
from ollama import chat

def evaluate_clinical_summaries(clinical_cases, discharge_summaries, output_file):
    """
    Evaluate clinical case summaries against discharge summaries.
    
    Args:
        clinical_cases (list): List of clinical case texts.
        discharge_summaries (list): List of discharge summary texts.
        output_file (str): Path to save the output dictionary.
        generate_score (callable): Function to generate scores using the AI model.
    
    Returns:
        None
    """
    gen_dictionary = {}
    
    for i in range(len(clinical_cases)):
        prompt = f"""Look at these guidelines carefully, i have also provided the dataset for you to analyze:

            Guidelines : {guidelines}

            {clinical_cases[i]}

            {discharge_summaries[i]}
            Instructions:

            using clinical case and discharge summary, evaluate and provide a score (1 to 5) for each feature listed above in the guidlines.
            Only provide numeric scores for each feature; do not include comments or explanations.
            Ensure that each score reflects a direct comparison of the clinical case and its corresponding discharge summary.
            Just provide the score for each feature, do not provide any additional information. 
            Do not include any comments or explanations. before or after the scores
            
            
            """
        
        
        # response = generate_score(prompt)
        stream = chat(
        model='llama3.2',
        messages=[{'role': 'user', 'content': prompt}],
        stream=True,
        )
        print(f"SCORE FOR SCALE FEATURES FOR CLINICAL CASE {i} AND DISCHARGE SUMMARY {i}:")
        for chunk in stream:
            content = chunk['message']['content']  
            print(content, end='', flush=True)
        gen_dictionary[i] = content

        if i % 10 == 0:
            print(f"Processed {i}/{len(clinical_cases)} records")
    
    with open(output_file, "w") as f:
        json.dump(gen_dictionary, f, indent=4)
    print(f"Results saved to {output_file}")


    
output_file = "output.txt"
    # Call the evaluation function
evaluate_clinical_summaries(clinical_cases, discharge_summaries, output_file)


SCORE FOR SCALE FEATURES FOR CLINICAL CASE 0 AND DISCHARGE SUMMARY 0:


KeyboardInterrupt: 